# Apply ARIMA to the Electricity Dataset
Assume user downloaded archive.zip from Kaggle,
renamed the file BuildingData.zip,
and stored the file in the data subdirectory.
Assume the zip file contains the electricity_cleaned.csv file.  

In [1]:
DATAPATH=''
try:
    # On Google Drive, set path to my drive / data directory.
    from google.colab import drive
    IN_COLAB = True
    PATH='/content/drive/'
    drive.mount(PATH)
    DATAPATH=PATH+'My Drive/data/'  # must end in "/"
except:
    # On home computer, set path to local data directory.
    IN_COLAB = False
    DATAPATH='C:/'  # must end in "/"

ZIP_FILE='BuildingData.zip'
ZIP_PATH = DATAPATH+ZIP_FILE
ELEC_FILE='electricity_cleaned.csv'
MODEL_FILE='Model'  # will be used later to save models

In [2]:
from os import listdir
import csv
from zipfile import ZipFile
import numpy as np
import pandas as pd
from pandas.plotting import autocorrelation_plot

from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import StandardScaler

from statsmodels.tsa.arima.model import ARIMA

import matplotlib.pyplot as plt
from matplotlib import colors
mycmap = colors.ListedColormap(['red','blue'])  # list color for label 0 then 1
np.set_printoptions(precision=2)

In [9]:
def read_zip_to_panda(zip_filename,csv_filename):
    zip_handle = ZipFile(zip_filename)
    csv_handle = zip_handle.open(csv_filename)
    panda = pd.read_csv(csv_handle)
    return panda
def fix_date_type(panda):
    temp_col = pd.to_datetime(panda['timestamp'])
    temp_tab = panda.drop(['timestamp'],axis=1)
    panda = temp_tab
    panda.insert(0,'date',temp_col,True)
    return panda
def get_building_timeseries(panda,building):
    # Assume the panda dataframe has a datetime column.
    # (If not, call fix_date_type() before this.)
    # Extract the timeseries for one site.
    # Convert the datetime column to a DatetimeIndex.
    building_df = list(elec_df)==building
    temp_col = building_df['date']
    temp_val = temp_col.values
    temp_ndx = pd.DatetimeIndex(temp_val)
    dropped = building_df.drop('date',axis=1)
    panda = dropped.set_index(temp_ndx)
    return panda

In [10]:
elec_df = read_zip_to_panda(ZIP_PATH,ELEC_FILE)
elec_df = fix_date_type(elec_df)
elec_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17544 entries, 0 to 17543
Columns: 1579 entries, date to Mouse_science_Micheal
dtypes: datetime64[ns](1), float64(1578)
memory usage: 211.3 MB


In [11]:
elec_df

,date,Panther_parking_Lorriane,Panther_lodging_Cora,Panther_office_Hannah,Panther_lodging_Hattie,Panther_education_Teofila,Panther_education_Jerome,Panther_retail_Felix,Panther_parking_Asia,Panther_education_Misty,...,Cockatoo_public_Caleb,Cockatoo_education_Tyler,Cockatoo_public_Shad,Mouse_health_Buddy,Mouse_health_Modesto,Mouse_lodging_Vicente,Mouse_health_Justin,Mouse_health_Ileana,Mouse_health_Estela,Mouse_science_Micheal
0,2016-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,123.200,727.575,69.200,8.8224,370.0870,10.0,282.9965,26.0,135.0,168.2243
1,2016-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,126.475,731.200,66.275,17.6449,737.8260,30.0,574.9265,51.0,265.0,336.4486
2,2016-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,127.825,724.675,64.675,17.6449,729.9255,30.0,570.2780,50.0,272.0,336.4486
3,2016-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,130.475,737.375,65.875,17.6449,722.2620,20.0,561.1470,52.0,276.0,336.4486
4,2016-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,129.675,721.150,66.275,17.6449,719.1665,30.0,564.3695,50.0,280.0,336.4486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17539,2017-12-31 19:00:00,15.4830,135.2261,3.4357,79.1353,105.6374,465.0898,67.0199,35.7069,16.3231,...,96.925,704.950,111.350,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17540,2017-12-31 20:00:00,12.7224,135.9262,3.4087,81.6958,107.7348,463.6895,56.6869,35.7069,16.0831,...,97.550,695.700,115.875,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17541,2017-12-31 21:00:00,11.2822,135.1761,3.3546,82.8160,106.1295,461.2890,55.0576,35.5068,16.1631,...,93.825,687.325,111.650,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17542,2017-12-31 22:00:00,16.9233,137.6266,3.2876,82.3359,109.6282,460.5889,49.6776,35.7069,14.8829,...,94.150,674.275,111.950,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
park_cols = [c for c in elec_df.columns if 'Gator' in c]
print(park_cols)

['Gator_public_Nettie', 'Gator_assembly_Selma', 'Gator_public_Marcie', 'Gator_assembly_Beryl', 'Gator_public_Lindsey', 'Gator_public_Clara', 'Gator_public_Marissa', 'Gator_public_Maude', 'Gator_assembly_Erich', 'Gator_assembly_Marjorie', 'Gator_public_Kenny', 'Gator_other_Reginald', 'Gator_assembly_Loyce', 'Gator_assembly_Blanca', 'Gator_office_Carrie', 'Gator_assembly_Maurine', 'Gator_public_Latasha', 'Gator_other_Cassandra', 'Gator_public_Dewey', 'Gator_assembly_Stacy', 'Gator_assembly_Lucia', 'Gator_assembly_Joni', 'Gator_public_Cheri', 'Gator_public_Everette', 'Gator_other_Glen', 'Gator_public_Beulah', 'Gator_assembly_Virgie', 'Gator_public_Geraldine', 'Gator_public_Jayme', 'Gator_office_August', 'Gator_assembly_Lera', 'Gator_assembly_Daisy', 'Gator_public_Erika', 'Gator_assembly_Elliot', 'Gator_assembly_Roy', 'Gator_public_Noe', 'Gator_public_Leroy', 'Gator_other_Minda', 'Gator_other_Elfriede', 'Gator_public_Natasha', 'Gator_public_Philip', 'Gator_assembly_Lelia', 'Gator_office_Li

In [6]:
Gator = elec_df.filter(like='Gator')
Gator 

,Gator_public_Nettie,Gator_assembly_Selma,Gator_public_Marcie,Gator_assembly_Beryl,Gator_public_Lindsey,Gator_public_Clara,Gator_public_Marissa,Gator_public_Maude,Gator_assembly_Erich,Gator_assembly_Marjorie,...,Gator_other_Gertrude,Gator_public_Ross,Gator_warehouse_Constance,Gator_public_Kendall,Gator_assembly_Lilli,Gator_public_Dale,Gator_public_Janna,Gator_warehouse_Stacie,Gator_other_Refugio,Gator_assembly_Gene
0,20.0000,1.6667,123.3333,0.5417,25.0000,15.9583,13.3333,108.3333,14.80,35.0417,...,0.6667,8.3333,15.0000,20.1667,3.7917,50.0000,91.6667,NaN,0.1250,29.1667
1,20.0000,1.6667,123.3333,0.5417,25.0000,15.9583,13.3333,108.3333,17.61,35.0417,...,0.6667,8.3333,15.0000,20.1667,3.7917,50.0000,91.6667,NaN,0.1250,29.1667
2,20.0000,1.6667,123.3333,0.5417,25.0000,15.9583,13.3333,108.3333,17.68,35.0417,...,0.6667,8.3333,15.0000,20.1667,3.7917,50.0000,91.6667,NaN,0.1250,29.1667
3,20.0000,1.6667,123.3333,0.5417,25.0000,15.9583,13.3333,108.3333,16.90,35.0417,...,0.6667,8.3333,15.0000,20.1667,3.7917,50.0000,91.6667,NaN,0.1250,29.1667
4,20.0000,1.6667,123.3333,0.5417,25.0000,15.9583,13.3333,108.3333,16.74,35.0417,...,0.6667,8.3333,15.0000,20.1667,3.7917,50.0000,91.6667,NaN,0.1250,29.1667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17539,21.6667,1.0417,76.6667,NaN,16.6667,11.3333,8.3333,104.1667,NaN,54.4583,...,1.0417,10.0000,6.6667,13.5000,1.6250,20.8333,133.3333,11.6667,0.0833,20.8333
17540,21.6667,1.0417,76.6667,NaN,16.6667,11.3333,8.3333,104.1667,NaN,54.4583,...,1.0417,10.0000,6.6667,13.5000,1.6250,20.8333,133.3333,11.6667,0.0833,20.8333
17541,21.6667,1.0417,76.6667,NaN,16.6667,11.3333,8.3333,104.1667,NaN,54.4583,...,1.0417,10.0000,6.6667,13.5000,1.6250,20.8333,133.3333,11.6667,0.0833,20.8333
17542,21.6667,1.0417,76.6667,NaN,16.6667,11.3333,8.3333,104.1667,NaN,54.4583,...,1.0417,10.0000,6.6667,13.5000,1.6250,20.8333,133.3333,11.6667,0.0833,20.8333


In [7]:
buildings= list(elec_df)


In [8]:
# Plot temperature time series per site. 
# Separate plots stacked vertically.
for building in buildings:
    building_df = get_building_timeseries(elec_df,building)
    #temp_df = building_df[Gator ]
    temp_df.plot(figsize=(20,5))
    plt.plot(elec_df[building])
    plt.title(building)
    plt.show()

TypeError: 'bool' object is not subscriptable

In [ ]:
# See the Electricity_100 notebook for basic stats.

In [ ]:
# Not done: start ARIMA analysis.